In [1]:
import pandas as pd
import numpy as np
from helpers import calibrate_train_clfs, load_data, get_baseline, print_results, run_clf

In [2]:
keep_columns =  ['IsTraining','FTR', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 
    'away_away_team_corners_against', 'away_away_team_corners_for',	'away_away_team_goals_against',	'away_away_team_goals_for',	
    'away_away_team_possession',	'away_away_team_shotsoff_against',	'away_away_team_shotsoff_for',	'away_away_team_shotson_against',
    	'away_away_team_shotson_for',	'away_direct_team_corners_against',	'away_direct_team_corners_for',	'away_direct_team_goals_against',
        'away_direct_team_goals_for',	'away_direct_team_possession',	'away_direct_team_shotsoff_against',	'away_direct_team_shotsoff_for',
        'away_direct_team_shotson_against',	'away_direct_team_shotson_for',	'home_direct_team_corners_against',	'home_direct_team_corners_for',	
        'home_direct_team_goals_against',	'home_direct_team_goals_for',	'home_direct_team_possession',	'home_direct_team_shotsoff_against',
        'home_direct_team_shotsoff_for',	'home_direct_team_shotson_against',	'home_direct_team_shotson_for',	
        'home_home_team_corners_against',	'home_home_team_corners_for',	'home_home_team_goals_against',	'home_home_team_goals_for',	
        'home_home_team_possession',	'home_home_team_shotsoff_against',	'home_home_team_shotsoff_for',	'home_home_team_shotson_against',
                 'home_home_team_shotson_for','Referee' 
]
 
drop_columns = [
    'B365H', 'B365D', 'B365A',
    'AC', 'AF', 'AR', 'AS', 'AST', 'AY', 
    'BWA', 'BWD', 'BWH', 'Bb1X2', 'BbAH', 'BbAHh', 'BbAv<2.5', 'BbAv>2.5', 'BbAvA', 'BbAvAHA', 
    'BbAvAHH', 'BbAvD', 'BbAvH', 'BbMx<2.5', 'BbMx>2.5', 'BbMxA', 'BbMxAHA', 'BbMxAHH', 'BbMxD',
    'BbMxH', 'BbOU', 'FTAG', 
    'HC', 'HF', 'HR', 'HS', 'HST', 
    'HTAG', 'HTHG', 'HTR', 'HY', 'IWA', 'IWD', 'IWH', 
    'LBA', 'LBD', 'LBH', 'PSA', 'PSCA', 'PSCD', 'PSCH', 'PSD', 'PSH', 
    'VCA', 'VCD', 'VCH', 'WHA', 'WHD', 'WHH', 'GoalFirstHalf', 'SHHG', 'SHAG', 'GoalSecondHalf','FTR','BothToScore',
]

#
target_column = "FTR"
drop_columns.remove(target_column)
X,y = load_data(columns_to_drop=drop_columns, is_training=True, target_name=target_column)

KEEEP
LENGTH1180


In [3]:
type(y)

pandas.core.series.Series

In [4]:
import numpy
import pandas
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import np_utils
from keras.layers import Bidirectional, LSTM
from keras.initializers import glorot_normal

from sklearn.model_selection import cross_val_score, train_test_split, TimeSeriesSplit
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [5]:
y=y.values
X=X.values

encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [6]:
from keras import backend as K
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(12,  activation='relu')) # input_dim=314,
#   model.add(Bidirectional(LSTM(256,  return_sequences=True, dropout=0.1, recurrent_dropout=0.1,kernel_initializer=glorot_normal(seed=None)),name = 'BDLSTM1')) #Best = 300,0.25,0.25
#    model.add(Dense(8, activation='relu'))
#    model.add(Dense(1, activation='linear'))
    model.add(Dense(3, activation='softmax'))
#     # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mae', 'mse', f1_m], verbose=1)
    #model.compile(loss='mse', optimizer='adam', metrics=['mse','mae']) 
    return model
estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=2, verbose=1)

In [7]:
# #seed = 7
# #numpy.random.seed(seed)
# from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)# max_train_size=100
# for train_index, test_index in tscv.split(X):
#     print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

# #kfold = KFold(n_splits=10)
# # shuffle=True, random_state=seed
results = cross_val_score(estimator, X, dummy_y, cv=tscv.split(X))

Epoch 1/10
295/295 [==============================] - 7s 25ms/step - loss: 2.6350 - acc: 0.3729 - mean_absolute_error: 0.4165 - mean_squared_error: 0.3120 - f1_m: 0.3593
Epoch 2/10
295/295 [==============================] - 2s 7ms/step - loss: 1.6631 - acc: 0.3695 - mean_absolute_error: 0.4319 - mean_squared_error: 0.2861 - f1_m: 0.3085
Epoch 3/10
295/295 [==============================] - 2s 7ms/step - loss: 1.3183 - acc: 0.3898 - mean_absolute_error: 0.4167 - mean_squared_error: 0.2515 - f1_m: 0.3085A: 0s - loss: 1.3638 - acc: 0.4225 - mean_absolute_error: 0.4021 - mean_squared_error: 0
Epoch 4/10
295/295 [==============================] - 2s 6ms/step - loss: 1.2416 - acc: 0.4339 - mean_absolute_error: 0.4049 - mean_squared_error: 0.2353 - f1_m: 0.3605
Epoch 5/10
295/295 [==============================] - 2s 6ms/step - loss: 1.2240 - acc: 0.4576 - mean_absolute_error: 0.3910 - mean_squared_error: 0.2324 - f1_m: 0.3774
Epoch 6/10
295/295 [==============================] - 2s 7ms/step 

In [8]:
results

array([0.54915254, 0.48474576, 0.6440678 ])

In [9]:
import statistics 
x = statistics.mean(results)
x

0.559322033898305

In [10]:
X_train_calibrate, X_test, y_train_calibrate, y_test = train_test_split(X, y, test_size=0.20)
X_train, X_calibrate, y_train, y_calibrate = train_test_split(X_train_calibrate, y_train_calibrate, test_size = 0.3, random_state = 42)

In [11]:
model = baseline_model()
model.fit(X, dummy_y, nb_epoch=30, batch_size=10, validation_split=0.2, verbose=1)

c:\users\martin\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 944 samples, validate on 236 samples
Epoch 1/30
944/944 [==============================] - 2s 2ms/step - loss: 4.2608 - acc: 0.4131 - mean_absolute_error: 0.3957 - mean_squared_error: 0.3338 - f1_m: 0.4067 - val_loss: 2.7519 - val_acc: 0.5169 - val_mean_absolute_error: 0.3372 - val_mean_squared_error: 0.2668 - val_f1_m: 0.5183
Epoch 2/30
944/944 [==============================] - 2s 2ms/step - loss: 1.8324 - acc: 0.4714 - mean_absolute_error: 0.3680 - mean_squared_error: 0.2650 - f1_m: 0.4586 - val_loss: 1.9954 - val_acc: 0.4492 - val_mean_absolute_error: 0.3822 - val_mean_squared_error: 0.2935 - val_f1_m: 0.4282
Epoch 3/30
944/944 [==============================] - 2s 2ms/step - loss: 1.5595 - acc: 0.4905 - mean_absolute_error: 0.3621 - mean_squared_error: 0.2502 - f1_m: 0.4751 - val_loss: 1.1822 - val_acc: 0.5720 - val_mean_absolute_error: 0.3341 - val_mean_squared_error: 0.2099 - val_f1_m: 0.5310
Epoch 4/30
944/944 [==============================] - 2s 2ms/step - loss: 1.30

Epoch 29/30
944/944 [==============================] - 2s 2ms/step - loss: 0.6666 - acc: 0.7066 - mean_absolute_error: 0.2424 - mean_squared_error: 0.1284 - f1_m: 0.6986 - val_loss: 0.8395 - val_acc: 0.6483 - val_mean_absolute_error: 0.2750 - val_mean_squared_error: 0.1591 - val_f1_m: 0.6289
Epoch 30/30
944/944 [==============================] - 2s 2ms/step - loss: 0.6831 - acc: 0.7066 - mean_absolute_error: 0.2395 - mean_squared_error: 0.1325 - f1_m: 0.7019 - val_loss: 1.0621 - val_acc: 0.5636 - val_mean_absolute_error: 0.2893 - val_mean_squared_error: 0.2007 - val_f1_m: 0.5654


In [12]:
prediction = model.predict(X[0:100])
prediction /= prediction.sum(axis=1).reshape((-1, 1))
prediction

array([[9.95674431e-01, 4.32524923e-03, 2.59739380e-07],
       [5.69902398e-02, 3.12833115e-02, 9.11726475e-01],
       [4.67384458e-02, 9.32029903e-01, 2.12316737e-02],
       [4.96488588e-04, 3.72496457e-03, 9.95778561e-01],
       [1.70534253e-01, 3.59123081e-01, 4.70342696e-01],
       [9.95621204e-01, 4.37571248e-03, 3.11455028e-06],
       [5.56902528e-01, 2.45877370e-01, 1.97220132e-01],
       [5.71253121e-01, 1.69066906e-01, 2.59679943e-01],
       [8.05065453e-01, 1.11366078e-01, 8.35685059e-02],
       [8.40273201e-01, 4.48878594e-02, 1.14838965e-01],
       [6.75323009e-01, 1.55406967e-01, 1.69270068e-01],
       [5.44393539e-01, 2.75240034e-01, 1.80366412e-01],
       [8.52769732e-01, 6.68345466e-02, 8.03956687e-02],
       [7.46895522e-02, 9.52452943e-02, 8.30065191e-01],
       [2.96437413e-01, 3.80091220e-01, 3.23471397e-01],
       [3.69575799e-01, 3.53354603e-01, 2.77069658e-01],
       [3.77992839e-01, 5.17454267e-01, 1.04552910e-01],
       [7.71432161e-01, 7.90895